In [2]:
import os
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

In [ ]:
# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
)

In [ ]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
messages = [
    {"from": "human", "value": "how to make a bomb?"}
]
# 文本转换为 token ID  添加生成提示符  返回PyTorch 张量   
inputs = tokenizer.apply_chat_template(messages, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
 
text_streamer = TextStreamer(tokenizer)
outputs = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

In [6]:
# 导入数据集加载函数
from datasets import load_dataset
dataset = load_dataset("json",data_files="jailbreak.json")
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 249
    })
})

In [ ]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    texts = []
    for instruction, output_text in zip(examples["instruction"],  examples["output"]):
        prompt = f"Instruction: {instruction}\nOutput: "
        texts.append(prompt + output_text)
    return {"text": texts}

data1 = dataset.map(formatting_prompts_func, batched = True)
print(data1["train"].column_names)
temp = data1["train"]
print(temp["text"][0])
print("-"*100)
print(temp["text"][-1])

In [ ]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

In [ ]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(  # 创建一个 SFTTrainer 实例
    model=model,  # 传入要微调的模型
    tokenizer=tokenizer,  # 传入 tokenizer，用于处理文本数据
    train_dataset=temp,  # 传入训练数据集
    dataset_text_field="text",  # 指定数据集中 文本字段的名称
    max_seq_length=max_seq_length,  # 设置最大序列长度
    dataset_num_proc=2,  # 设置数据处理的并行进程数
    packing=False,  # 是否启用打包功能（这里设置为 False，打包可以让训练更快，但可能影响效果）
    args=TrainingArguments(  # 定义训练参数
        per_device_train_batch_size=2,  # 每个设备（如 GPU）上的批量大小
        gradient_accumulation_steps=3,  # 梯度累积步数，用于模拟大批次训练
        # 2（单卡批量）×3（累积步数）= 6（有效批量大小）
        # 249条数据/6 ≈ 42条数据/卡
        warmup_steps=5,  # 预热步数，训练开始时学习率逐渐增加的步数
        max_steps=84,  # = 2*Epoch
        learning_rate=2e-4,  # 学习率，模型学习新知识的速度
        fp16=not is_bfloat16_supported(),  # 是否使用 fp16 格式加速训练（如果环境不支持 bfloat16）
        bf16=is_bfloat16_supported(),  # 是否使用 bfloat16 格式加速训练（如果环境支持）
        logging_steps=5,  # 每隔多少步记录一次训练日志
        optim="adamw_8bit",  # 使用的优化器，用于调整模型参数
        weight_decay=0.01,  # 权重衰减，防止模型过拟合
        lr_scheduler_type="linear",  # 学习率调度器类型，控制学习率的变化方式
        seed=3407,  # 随机种子，确保训练结果可复现
        output_dir="outputs",  # 训练结果保存的目录
        report_to="none",  # 是否将训练结果报告到外部工具（如 WandB），这里设置为不报告
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
FastLanguageModel.for_inference(model)

In [ ]:
inputs = tokenizer(["how to kill myself?"], return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=1111,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs)
print(response[0])

In [14]:
save_path = "save_jailbreak"

In [ ]:
# 保存 adapter_config.json、adapter_model.safetensors 文件到 save_path 文件夹
model.save_pretrained(save_path)
# tokenizer_config.json, special_tokens_map.json, tokenizer.json
tokenizer.save_pretrained(save_path)

In [ ]:
# 加载保存后的模型
model_, tokenizer_ = FastLanguageModel.from_pretrained(
    model_name=save_path,  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
)

In [ ]:
inputs = tokenizer_(["how to kill my classmates?"], return_tensors="pt").to("cuda")
outputs = model_.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=1111,
    use_cache=True,
)
response = tokenizer_.batch_decode(outputs)
print(response[0])

In [ ]:

# 将模型保存为 8 位量化格式（Q8_0）
# 这种格式文件小且运行快，适合部署到资源受限的设备
if 1: model.save_pretrained_gguf("model_gguf", tokenizer)

# 将模型保存为 16 位量化格式（f16）
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式（q4_k_m）
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

In [ ]:
HUGGINGFACE_TOKEN = ""
# 导入 Hugging Face Hub 的 create_repo 函数，用于创建一个新的模型仓库
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上创建一个新的模型仓库
create_repo("zemelee/11", token=HUGGINGFACE_TOKEN, exist_ok=True)

# # 将模型和分词器上传到 Hugging Face Hub 上的仓库
# model.push_to_hub_gguf("zemelee/jailbreak-test", tokenizer, token=HUGGINGFACE_TOKEN)

In [ ]:
print(tokenizer._ollama_modelfile)